### Analyse et comprehension des fichiers

In [1]:
import matplotlib.pyplot as plt
from read_data import *
from utils import *
from fasttext_utils import *
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor

In [2]:
store_full_dataset_with_features(from_scratch=False, vectorize=False)

/tmp/ipykernel_3261/2814915721.py:1: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  store_full_dataset_with_features(from_scratch=False, vectorize=False)


Starting data columns : ['author', 'hindex', 'text', 'nb_paper', 'modindx', 'hindex_lab', 'core_number', 'pagerank', 'authority', 'n_neighbors_dist_1', 'min_neighbors_dist_1', 'mean_neighbors_dist_1', 'max_neighbors_dist_1', 'n_neighbors_dist_2', 'min_neighbors_dist_2', 'mean_neighbors_dist_2', 'max_neighbors_dist_2', 'clustering_coef', 'vector_coord_0', 'vector_coord_1']
Ending data columns : ['author', 'hindex', 'text', 'nb_paper', 'modindx', 'hindex_lab', 'core_number', 'pagerank', 'authority', 'n_neighbors_dist_1', 'min_neighbors_dist_1', 'mean_neighbors_dist_1', 'max_neighbors_dist_1', 'n_neighbors_dist_2', 'min_neighbors_dist_2', 'mean_neighbors_dist_2', 'max_neighbors_dist_2', 'clustering_coef', 'vector_coord_0', 'vector_coord_1']


In [3]:
train, test = get_processed_data()

/tmp/ipykernel_3261/1429269039.py:1: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  train, test = get_processed_data()


**distribution des données**

In [ ]:
train,n_train = get_train_data()
test,n_test = get_test_data()

In [ ]:
plt.figure(figsize=(10,5))
plt.hist(x=train['hindex'],bins=187);

**exploitation de model**

On est sur que le modèle applique softmax sur la représentation du mot. Nous ou choisit donc d'appliquer un autre régresseur

**Data for multiple regression model**

In [6]:
X_train, y_train, X_test, y_test = get_numpy_data(n=174241)

/tmp/ipykernel_3261/2692092469.py:1: DtypeWarning: Columns (5) have mixed types.Specify dtype option on import or set low_memory=False.
  X_train, y_train, X_test, y_test = get_numpy_data(n=174241)


In [7]:
X_train_norm, X_test_norm = normalize(X_train, X_test)

**Random forest**

In [10]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

**XG_BOOST**

In [ ]:
model_xg = XGBRegressor(learning_rate=0.01, max_depth=5, objective="reg:linear", subsample=0.7, n_estimators=500)
model_xg.fit(X_train,y_train)
mod_preds = model_xg.predict(X_test)
mean_squared_error(y_test,mod_preds)

**Neural Network**

In [ ]:

from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
model_nn = keras.Sequential()
model_nn.add(layers.Dense(64, kernel_initializer='uniform', input_shape=(10,)))
model_nn.add(layers.Activation('softmax'))

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-2,
    decay_steps=10000,
    decay_rate=0.9)
optimizer = keras.optimizers.SGD(learning_rate=lr_schedule)

model_nn.compile(loss='mse', optimizer=optimizer)

model_nn.fit(X_train,y_train, epochs=10, batch_size=5)
mod_nn_preds = model_nn.predict(X_test)
print(len(mod_nn_preds))
mean_squared_error(y_test,mod_nn_preds)

**Lasso linear**

In [ ]:
from sklearn import linear_model
model_lasso = linear_model.Lasso(alpha=0.75, fit_intercept=False, tol=0.000000001,max_iter=10000000, positive=True)
model_lasso.fit(X_train,y_train)
mod_lasso_preds = model_lasso.predict(X_test)
mean_squared_error(y_test,mod_lasso_preds)

**Decision tree**

In [ ]:
from sklearn import tree
model_tree = tree.DecisionTreeRegressor()
model_tree.fit(X_train,y_train)
mod_tree_preds = model_tree.predict(X_test)
mean_squared_error(y_test,mod_tree_preds)

### CATBOOST

In [11]:
from catboost import CatBoostRegressor
model_cat = CatBoostRegressor(verbose=False)
model_cat.fit(X_train, y_train)
mod_preds = model_cat.predict(X_test)
mean_squared_error(y_test,mod_preds)

48.49073976744716

**Light GBM**

In [12]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label = y_train)
params = {}
params['learning_rate'] = 0.100
params['num_iterations'] = 500
params['boosting_type'] = 'dart'
params['objective'] = 'regression'
params['metric'] = 'rmse'
params['sub_feature'] = 0.65
params['num_leaves'] = 50
params['min_data'] = 60
params['max_depth'] = 30
clf = lgb.train(params, d_train, 100)
y_pred=clf.predict(X_test)
mean_squared_error(y_test,y_pred)

/home/tristan/.local/lib/python3.9/site-packages/lightgbm/engine.py:177: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.008770 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2830
[LightGBM] [Info] Number of data points in the train set: 130680, number of used features: 15
[LightGBM] [Info] Start training from score 10.071786


48.5970961037371

**test linear regression**

In [ ]:
from sklearn import linear_model

reg = linear_model.TweedieRegressor().fit(X_train,y_train)

In [ ]:
reg_pred = reg.predict(X_test)

In [ ]:
mean_squared_error(y_test,reg_pred)

**test svm**

In [ ]:
from sklearn.svm import SVC

model_svm =SVC(kernel='sigmoid', C=2)

In [ ]:
model_svm.fit(X_train,y_train)

In [ ]:
svm_pred = model_svm.predict(X_test)

In [ ]:
mean_squared_error(y_test,svm_pred)

**Test CNN**

# Soumission

In [ ]:
from submit import submit
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
# from catboost import CatBoostRegressor

model_cat = CatBoostRegressor(verbose=False)
model_forest = RandomForestRegressor()
model_xg = XGBRegressor(learning_rate=0.01, max_depth=5, objective="reg:linear", subsample=0.7, n_estimators=500)

submit(model_cat)